<a href="https://colab.research.google.com/github/basharEg/dl_hw_1/blob/main/Fizzbuzz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


Instructions for updating:
non-resource variables are not supported in the long term


# Represent each input by an array of its binary digits.


In [ ]:
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])


# One-hot encode the desired outputs: [number, "fizz", "buzz", "fizzbuzz"]


In [ ]:
def fizz_buzz_encode(i):
    if   i % 15 == 0: return np.array([0, 0, 0, 1])
    elif i % 5  == 0: return np.array([0, 0, 1, 0])
    elif i % 3  == 0: return np.array([0, 1, 0, 0])
    else:             return np.array([1, 0, 0, 0])

# Our goal is to produce fizzbuzz for the numbers 1 to 100. So it would be
# unfair to include these in our training data. Accordingly, the training data
# corresponds to the numbers 101 to (2 ** NUM_DIGITS - 1).



In [ ]:
NUM_DIGITS = 10
trX = np.array([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = np.array([fizz_buzz_encode(i)          for i in range(101, 2 ** NUM_DIGITS)])

# Our variables. The input has width NUM_DIGITS, and the output has width 4.


In [ ]:

X = tf.placeholder(tf.float32, [None, NUM_DIGITS])
Y = tf.placeholder(tf.float32, [None, 4])

# How many units in the hidden layer.


In [ ]:
NUM_HIDDEN = 100


# We'll want to randomly initialize weights.


In [ ]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

# Initialize the weights.


In [ ]:
w_h = init_weights([NUM_DIGITS, NUM_HIDDEN])
w_o = init_weights([NUM_HIDDEN, 4])

# Our model is a standard 1-hidden-layer multi-layer-perceptron with ReLU
# activation. The softmax (which turns arbitrary real-valued outputs into
# probabilities) gets applied in the cost function.

In [ ]:
def model(X, w_h, w_o):
    h = tf.nn.relu(tf.matmul(X, w_h))
    return tf.matmul(h, w_o)

# Predict y given x using the model.


In [ ]:
py_x = model(X, w_h, w_o)

# We'll train our model by minimizing a cost function.


In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=py_x))
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost)

# And we'll make predictions by choosing the largest output.


In [ ]:
predict_op = tf.argmax(py_x, 1)


# Finally, we need a way to turn a prediction (and an original number)
# into a fizz buzz output

In [ ]:
def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

# Launch the graph in a session


In [ ]:
BATCH_SIZE = 128
# Launch the graph in a session
with tf.Session() as sess:
    tf.initialize_all_variables().run()

    for epoch in range(10000):
        # Shuffle the data before each training iteration.
        p = np.random.permutation(range(len(trX)))
        trX, trY = trX[p], trY[p]

        # Train in batches of 128 inputs.
        for start in range(0, len(trX), BATCH_SIZE):
            end = start + BATCH_SIZE
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})

        # And print the current accuracy on the training data.
        print(epoch, np.mean(np.argmax(trY, axis=1) ==
                             sess.run(predict_op, feed_dict={X: trX, Y: trY})))

    # And now for some fizz buzz
    numbers = np.arange(1, 101)
    teX = np.transpose(binary_encode(numbers, NUM_DIGITS))

    teY = sess.run(predict_op, feed_dict={X: teX})
    output = np.vectorize(fizz_buzz)(numbers, teY)

    print(output)

Streaming output truncated to the last 5000 lines.
5009 0.9891657638136512
5010 0.9902491874322861
5011 0.991332611050921
5012 0.9783315276273022
5013 0.9924160346695557
5014 0.9891657638136512
5015 0.9902491874322861
5016 0.9902491874322861
5017 0.9826652221018418
5018 0.9880823401950163
5019 0.9880823401950163
5020 0.9891657638136512
5021 0.9891657638136512
5022 0.9880823401950163
5023 0.9869989165763814
5024 0.9891657638136512
5025 0.991332611050921
5026 0.991332611050921
5027 0.9859154929577465
5028 0.9859154929577465
5029 0.9902491874322861
5030 0.9902491874322861
5031 0.9924160346695557
5032 0.9880823401950163
5033 0.991332611050921
5034 0.980498374864572
5035 0.9924160346695557
5036 0.991332611050921
5037 0.9924160346695557
5038 0.9902491874322861
5039 0.9902491874322861
5040 0.991332611050921
5041 0.991332611050921
5042 0.991332611050921
5043 0.9848320693391116
5044 0.9902491874322861
5045 0.9837486457204767
5046 0.991332611050921
5047 0.9902491874322861
5048 0.991332611050921
